# **🏠 네이버 부동산 크롤링**
> 더 자세한 정보가 나와있는 네이버 부동산 페이지를 크롤링하고자 한다.
> 기본 데이터에 포함되어 있지 않은 최대층, 저층기준, 앞서 직방에서 나오지 않았던 방개수, 화장실개수, 세대수, 주차면적 등 여러 정보를 가져올 수 있다.

## Contents
- Library Import
- Data Load
- Crawling of Naver real estate home page


## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

: 

## 2. Data Load

- 기존에 하던 데이터는 2007년 부터 이루어져 있기 때문에 절단해서 2017년부터 데이터에 포함하기로 결정

In [2]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
add = pd.read_csv('../Data/address_to_latlon_complete.csv')
train = pd.read_csv('../Data/KOREAN_train4.csv')
test = pd.read_csv('../Data/KOREAN_test4.csv')

In [3]:
train = train[train['계약년월'] >= 201701]
train = train[train['target'] > 6111.11]
train[['연', '월']] = train['계약년월'].apply(lambda x: list(divmod(x, 100))).tolist()

# 정규표현식 패턴 생성
excluded_apts = ['AirPalace', 'SRvill', '경동팰리스힐', '대길B', '썬앤빌', '재선주택', '코원']

# go
pattern = '|'.join(excluded_apts)

# '열이름' 열에서 패턴이 있는지 확인하여 새로운 열 추가
train['is_아파트'] = np.where(train['아파트명'].str.contains(pattern), 1, 0)

train = train[train['is_아파트'] == 0]

train = train.drop(columns='is_아파트')

In [4]:
apart = train['아파트명'].unique()

In [5]:
train[['시', '구', '동']] = train['시군구'].str.split(expand = True)
test[['시', '구', '동']] = test['시군구'].str.split(expand = True)

In [6]:
dap = train[['동', '아파트명']].drop_duplicates(['아파트명']).reset_index()
dap2 = test[['동', '아파트명']].drop_duplicates(['아파트명']).reset_index()

In [7]:
dap['동아파트명'] = dap['동'] + ' ' + dap['아파트명']
dap2['동아파트명'] = dap2['동'] + ' ' + dap2['아파트명']

In [8]:
dap_merge = pd.concat([dap['동아파트명'], dap2['동아파트명']], axis = 0)

In [9]:
d = dap_merge.reset_index()['동아파트명']

In [55]:
d[d == "신당동 태원팰리스5차"]

4608    신당동 태원팰리스5차
Name: 동아파트명, dtype: object

In [66]:
apt_info = pd.read_csv('../Data/아파트정보.csv')
apt_area = pd.read_csv('../Data/아파트면적정보.csv')
apt_info = apt_info.iloc[:4577, :]
apt_info

,동아파트명,세대수,저_최고층,총주차대수,용적률,건폐율,건설사
0,개포동 개포6차우성,270세대(총8개동),5층/5층,270대(세대당 1대),106%,21%,우성건설(주)
1,개포동 개포우성3차,405세대(총5개동),15층/15층,450대(세대당 1.11대),179%,15%,우성건설(주)
2,개포동 개포자이,NaN,NaN,NaN,NaN,NaN,NaN
3,개포동 개포주공1단지,NaN,NaN,NaN,NaN,NaN,NaN
4,개포동 개포주공6단지,1060세대(총9개동),13층/15층,300대(세대당 0.28대),-,-,대한주택공사
...,...,...,...,...,...,...,...
4572,신당동 약수하이츠,"2282세대(기타임대 684세대 포함, 총19개동)",8층/20층,2091대(세대당 0.91대),255%,19%,동아건설산업(주)
4573,신당동 인성아트빌,NaN,NaN,NaN,NaN,NaN,NaN
4574,신당동 진달래,19세대(총1개동),4층/5층,21대(세대당 1.1대),257%,58%,이다종합건설(주)
4575,신당동 청구e편한세상,"895세대(영구임대 158세대 포함, 총15개동)",7층/15층,1010대(세대당 1.12대),224%,23%,대림산업


## 3. Crawling

In [57]:
ChromeDriverManager().install()
browser = webdriver.Chrome() # 우리가 컨트롤 할 수 있는 브라우저가 실행이된다.

In [58]:
browser.get('https://new.land.naver.com/search?ms=37.4848041,127.0730056,16&a=APT:ABYG:JGC:PRE&e=RETAIL')

In [12]:
# # 열 이름을 리스트로 지정
# column_names = ['동아파트명', '세대수', '저_최고층',
#                  '총주차대수', '용적률', '건폐율', '건설사']

# area_names = ['동아파트명', '공급전용면적', '방수욕실수',
#               '해당면적세대수', '현관구조']


# # 빈 데이터프레임 생성
# apt_info = pd.DataFrame(columns=column_names)
# apt_area = pd.DataFrame(columns=area_names)

In [68]:
# 없는 인덱스번호 저장
idx = []

for c, apt in enumerate(d[4609:6200]):

    # 체크
    comp = 0


    try:
        # 검색어 입력
        browser.find_element(By.ID, 'land_search').click()
        browser.find_element(By.ID, 'land_search').send_keys(Keys.CONTROL + "a")
        browser.find_element(By.ID, 'land_search').send_keys(apt)
        browser.find_element(By.ID, 'land_search').send_keys(Keys.ENTER)


        time.sleep(0.8)

        # 단지정보 클릭
        browser.find_element(By.CLASS_NAME, 'complex_link').click()

        comp = 1

        time.sleep(0.8)

        # 세대수
        a1 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td[1]').text

        # 저/최고층
        a2 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td[2]').text

        # 총 주차대수
        a3 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[2]/td[2]').text

        # 용적률
        a4 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[3]/td[1]').text
        
        # 건폐율
        a5 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[3]/td[2]').text

        # 건설사
        a6 = browser.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[4]/td[1]').text

        # 데이터프레임에 집어넣기
        apt_info.loc[len(apt_info), :] = [apt, a1, a2, a3, a4, a5, a6]

        try:
            i = 0
            while True:
                
                time.sleep(0.8)
                
                # 단지내 면적별 정보 클릭
                browser.find_element(By.XPATH, f'//*[@id="tab{i}"]').click()
                
                time.sleep(0.8)
                
                comp = 2

                # 공급/전용
                b1 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[1]').text

                # 방수/욕실수
                b2 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[2]').text
                        
                # 해당면적 세대수
                b3 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[3]').text

                # 현관구조
                b4 = browser.find_element(By.XPATH, '//*[@id="tabpanel"]/table/tbody/tr[4]').text

                # 데이터에 집어넣기
                apt_area.loc[len(apt_area), :] = [apt, b1, b2, b3, b4]

                i += 1
        except:
            print(f"{apt}의 정보 담기를 {i}번까지 완료하였습니다.")
            pass

    except:
        if comp == 0:
            apt_info.loc[len(apt_info), :] = [apt, '', '', '', '', '', '']
            print(f"{apt}의 검색 결과가 하나가 아니거나 없습니다.")
            idx.append(c)

신당동 프라임1의 정보 담기를 7번까지 완료하였습니다.
신당동 프라임2의 검색 결과가 하나가 아니거나 없습니다.
신당동 하이의 검색 결과가 하나가 아니거나 없습니다.
신당동 하이1의 검색 결과가 하나가 아니거나 없습니다.
예장동 예장동삼익의 정보 담기를 7번까지 완료하였습니다.
을지로5가 삼성파크빌의 검색 결과가 하나가 아니거나 없습니다.
인현동2가 인현상가의 정보 담기를 7번까지 완료하였습니다.
장충동1가 장충동라임카운티의 정보 담기를 6번까지 완료하였습니다.
장충동1가 장충레지던스의 정보 담기를 7번까지 완료하였습니다.
장충동2가 장충의 검색 결과가 하나가 아니거나 없습니다.
중림동 브라운스톤서울의 정보 담기를 7번까지 완료하였습니다.
중림동 삼성사이버빌리지의 정보 담기를 7번까지 완료하였습니다.
중림동 성요셉의 정보 담기를 7번까지 완료하였습니다.
충무로4가 진양상가의 정보 담기를 7번까지 완료하였습니다.
충무로5가 충무로엘크루메크로시티2차의 검색 결과가 하나가 아니거나 없습니다.
필동1가 하니맨션의 정보 담기를 5번까지 완료하였습니다.
황학동 DUO302의 검색 결과가 하나가 아니거나 없습니다.
황학동 골드캐슬Ⅲ의 검색 결과가 하나가 아니거나 없습니다.
황학동 그랜드힐오피스텔의 정보 담기를 6번까지 완료하였습니다.
황학동 황학아크로타워의 정보 담기를 7번까지 완료하였습니다.
회현동1가 남산롯데캐슬아이리스의 정보 담기를 1번까지 완료하였습니다.
회현동2가 쌍용남산플래티넘의 정보 담기를 6번까지 완료하였습니다.
흥인동 동대문와이즈캐슬의 검색 결과가 하나가 아니거나 없습니다.
흥인동 청계천두산위브더제니스의 검색 결과가 하나가 아니거나 없습니다.
망우동 예성그린캐슬의 검색 결과가 하나가 아니거나 없습니다.
면목동 면목삼익의 정보 담기를 4번까지 완료하였습니다.
면목동 용마금호타운의 정보 담기를 1번까지 완료하였습니다.
면목동 용마동아의 정보 담기를 2번까지 완료하였습니다.
면목동 용마산금호어울림의 정보 담기를 5번까지 완료하였습니다.
면목동 용마산하늘채의 정보

In [74]:
apt_info

동아파트명       0
세대수      1371
저_최고층    1371
총주차대수    1371
용적률      1371
건폐율      1371
건설사      1371
dtype: int64

In [75]:
apt_area

동아파트명      0
공급전용면적     0
방수욕실수      0
해당면적세대수    0
현관구조       0
dtype: int64

In [73]:
idx

[1,
 2,
 3,
 5,
 9,
 14,
 16,
 17,
 22,
 23,
 24,
 30,
 31,
 33,
 35,
 36,
 37,
 40,
 41,
 42,
 45,
 46,
 52,
 60,
 61,
 62,
 63,
 66,
 68,
 72,
 73,
 84,
 86,
 91,
 94,
 95,
 98,
 101,
 106,
 109,
 116,
 122,
 126,
 127,
 129,
 130,
 133,
 135,
 139,
 140,
 149,
 159,
 160,
 161,
 174,
 176,
 186,
 187,
 188,
 192,
 193,
 199,
 205,
 206,
 212,
 215,
 216,
 221,
 222,
 225,
 226,
 231,
 233,
 235,
 242,
 244,
 246,
 248,
 256,
 257,
 260,
 261,
 262,
 263,
 265,
 266,
 267,
 271,
 273,
 276,
 277,
 278,
 279,
 282,
 287,
 288,
 289,
 291,
 293,
 294,
 295,
 296,
 297,
 298,
 300,
 310,
 311,
 312,
 313,
 315,
 316,
 319,
 322,
 347,
 348,
 351,
 354,
 355,
 358,
 359,
 360,
 361,
 362,
 363,
 366,
 381,
 382,
 391,
 402,
 403,
 404,
 407,
 425,
 430,
 433,
 439,
 448,
 451,
 457,
 458,
 470,
 471,
 475,
 477,
 480,
 484,
 485,
 488,
 490,
 491,
 503,
 506,
 507,
 511,
 513,
 515,
 517,
 518,
 519,
 520,
 523,
 524,
 526,
 539,
 540,
 541,
 542,
 545,
 546,
 547,
 549,
 550,
 551,
 553

In [72]:
# 저장
apt_info.to_csv('../Data/아파트정보.csv', index = False)
apt_area.to_csv('../Data/아파트면적정보.csv', index = False)